In [1]:
# Read namespaces
import os
import kubernetes
import ipywidgets as widgets
from IPython.display import display, JSON

ns_list = []
kubernetes.config.load_kube_config()
k8s_client = kubernetes.client.CoreV1Api()
namespaces = k8s_client.list_namespace(_request_timeout=(1, 5))
for namespace in namespaces.items:
    ns_name = namespace.metadata.name
    ns_list.append(ns_name)
w = widgets.Dropdown(
    options=ns_list,
    description='SUT ns',
    disabled=False,
    layout=widgets.Layout(width='50%')
)
display(w)

Dropdown(description='SUT ns', layout=Layout(width='50%'), options=('binderhub', 'calico-apiserver', 'calico-o…

In [6]:
# Set namespaces
SUT_NAMESPACE = w.value
# print(f"Using SUT namespace {SUT_NAMESPACE}")
BRANCH_NAME = SUT_NAMESPACE.replace("ci-ska-mid-itf-", "")
# print(f"Using SUT branch {BRANCH_NAME}")
SKA001_NAMESPACE = f"ci-dish-lmc-ska001-{BRANCH_NAME}"
# print(f"Using SKA 001 namespace {SKA001_NAMESPACE}")
ska001 = widgets.Text(
    value=SKA001_NAMESPACE,
    description='SKA001 ns',
    disabled=True,
    layout=widgets.Layout(width='60%')
)
display(ska001)
SKA036_NAMESPACE = f"ci-dish-lmc-ska036-{BRANCH_NAME}"
ska036 = widgets.Text(
    value=SKA036_NAMESPACE,
    description='SKA036 ns',
    disabled=True,
    layout=widgets.Layout(width='60%')
)
display(ska036)
# print(f"Using SKA 036 namespace {SKA036_NAMESPACE}")

# print(f"Setting up parameters for SUT namespace {SUT_NAMESPACE}\n")

TANGO_HOST = f"tango-databaseds.{SUT_NAMESPACE}.svc.miditf.internal.skao.int:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

th = widgets.Text(
    value=os.getenv("TANGO_HOST"),
    description='Tango host',
    disabled=True,
    layout=widgets.Layout(width='80%')
)
display(th)

Text(value='ci-dish-lmc-ska001-ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks', description='SKA001 …

Text(value='ci-dish-lmc-ska036-ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks', description='SKA036 …

Text(value='tango-databaseds.ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks.svc.miditf.internal.skao…

In [7]:
# Set Up and Configure Device Proxies

RECEPTORS = ["SKA001", "SKA036"]

# Config files set up
DATA_DIR = "../../data"
TMC_CONFIGS = f"{DATA_DIR}/mid_telescope/tmc"
SCAN_FILE = f"{TMC_CONFIGS}/scan.json"
RELEASE_RESOURCES_FILE = f"{DATA_DIR}/release_resources.json"

ASSIGN_RESOURCES_FILE = f"{TMC_CONFIGS}/assign_resources.json"
CONFIGURE_SCAN_FILE = f"{TMC_CONFIGS}/configure_scan.json"

CBF_CONFIGS = f"{DATA_DIR}/mid_telescope/cbf"
DISH_CONFIG_FILE = f"{CBF_CONFIGS}/sys_params/load_dish_config.json"

# print("Links for Taranta and QA Display\n")
print("SUT Links")
print(f"https://k8s.miditf.internal.skao.int/{SUT_NAMESPACE}/signal/display/")
print(f"https://k8s.miditf.internal.skao.int/{SUT_NAMESPACE}/taranta/devices")
print("\n")
print("Dish LMC links")
print(f"https://k8s.miditf.internal.skao.int/{SKA001_NAMESPACE}/taranta/devices")
print(f"https://k8s.miditf.internal.skao.int/{SKA036_NAMESPACE}/taranta/devices")

SUT Links
https://k8s.miditf.internal.skao.int/ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks/signal/display/
https://k8s.miditf.internal.skao.int/ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks/taranta/devices


Dish LMC links
https://k8s.miditf.internal.skao.int/ci-dish-lmc-ska001-ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks/taranta/devices
https://k8s.miditf.internal.skao.int/ci-dish-lmc-ska036-ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks/taranta/devices


In [8]:
import json
import time
import tango

from tango import DeviceProxy

# print(f'Setting up device proxies for {}')

try:
    # TMC proxies
    tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
    tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
    tmc_csp_subarray = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
    tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")

    # CSP.LMC proxies
    csp_control = DeviceProxy("mid-csp/control/0")
    csp_subarray = DeviceProxy("mid-csp/subarray/01")

    # CBF proxies
    cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
    cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")

    # Dish Leaf Proxies
    dish_leaf_node_ska001 = DeviceProxy("ska_mid/tm_leaf_node/d0001")
    dish_leaf_node_ska036 = DeviceProxy("ska_mid/tm_leaf_node/d0036")

    # SDP Proxies
    sdp_subarray = DeviceProxy("mid-sdp/subarray/01")
    

except tango.DevFailed as terr:
    print(f"ERROR: {terr.args[0].desc.strip()}")
    tmc_central_node = None
    tmc_csp_master = None
finally:
    print('Set up device proxies')




ERROR: device ska_mid/tm_central/central_node not defined in the database !
Set up device proxies


In [4]:
# Smoke test
if tmc_central_node is None:
    print(f"ERROR: No TMC central none")
else:
    tping = tmc_central_node.ping()
    print(f"Ping TMC central node: {tping}")
    tmc_central_node.adminMode = 0
    tmc_cent = widgets.Valid(
        value=not tmc_central_node.adminMode,
        description="TMC Central Node online",
        layout=widgets.Layout(width='60%'),
        style = {'description_width': 'initial'}
    )
    display(tmc_cent)

NameError: name 'tmc_central_node' is not defined

In [34]:
# Print TMC diagnostics
from IPython.display import Markdown
tmc_rows = ""
try:
    tmc_rows += f"| TMC Central Node | {tmc_central_node.name()} |\n"
    tmc_rows += f"| TMC Central Node state | {tmc_central_node.State()} |\n"
    tmc_rows += f"| TMC Central Node adminMode | {str(tmc_central_node.adminmode)} |\n"
    # print(f"TMC Central Node healthState: {str(tmc_central_node.health_state)}")
    # print(f"TMC Central Node telescopeHealthState: {str(tmc_central_node.telescope_health_state)}")
    # print(f"TMC Central Node isDishVccConfig: {str(tmc_central_node.isDishVccConfigSet)}")
    # print(f"TMC Central Node dishvccvalidationstatus: {str(tmc_central_node.dishvccvalidationstatus)}")
    # print(f"TMC Subarray Node state: {tmc_csp_subarray.State()}")
    # print(f"TMC Subarray adminMode: {str(tmc_csp_subarray.admin_mode)}")
    # print(f"TMC Subarray Node obsState: {str(tmc_csp_subarray.obs_state)}")
except tango.ConnectionFailed as terr:
    print(f"Tango connection failed: {terr.args[0].desc.strip()}")
Markdown(tmc_rows)
# table(tmc_rows, frame=True)
print("Done")

Done


In [18]:
# Print CSP-LMC Diagnostics
try:
    # Controller
    print(f"CSP-LMC Controller {csp_control.name()}")
    print(f"CSP-LMC Controller admin mode: {str(csp_control.adminMode)}")
    print(f"CSP-LMC Controller State: {csp_control.State()}")
    print(f"CSP-LMC Controller dish Vcc Config: {csp_control.dishVccConfig}")
    print(f"CSP-LMC Controller CBF Simulation Mode: {csp_control.cbfSimulationMode}")
    # Subarray
    print(f"CSP-LMC Subarray {csp_subarray.name()}")
    print(f"CSP-LMC Subarray admin mode: {str(csp_subarray.adminMode)}")
    print(f"CSP-LMC Subarray State: {csp_subarray.State()}")
    print(f"CSP-LMC Subarray observation state: {str(csp_subarray.obsState)}")
    print(f"CSP-LMC Subarray dish Vcc Config: {csp_subarray.dishVccConfig}")
except tango.ConnectionFailed as terr:
    print(f"Tango connection failed: {terr.args[0].desc.strip()}")


CSP-LMC Controller mid-csp/control/0
CSP-LMC Controller admin mode: adminMode.OFFLINE
CSP-LMC Controller State: DISABLE
CSP-LMC Controller dish Vcc Config: 
CSP-LMC Controller CBF Simulation Mode: True
CSP-LMC Subarray mid-csp/subarray/01
CSP-LMC Subarray admin mode: adminMode.OFFLINE
CSP-LMC Subarray State: DISABLE
CSP-LMC Subarray observation state: obsState.EMPTY
CSP-LMC Subarray dish Vcc Config: 


In [ ]:
# Set devices to adminMode = ONLINE
csp_control.adminMode = 0
csp_subarray.adminMode = 0

# Leaf Nodes
csp_subarray_leaf_node = "ska_mid/tm_leaf_node/csp_subarray01"
sdp_subarray_leaf_node = "ska_mid/tm_leaf_node/sdp_subarray01"
csp_master_leaf_node = "ska_mid/tm_leaf_node/csp_master"
csp_subarray_leaf_node_dp = DeviceProxy(csp_subarray_leaf_node)
sdp_subarray_leaf_node_dp = DeviceProxy(sdp_subarray_leaf_node)
csp_master_leaf_node_dp = DeviceProxy(csp_master_leaf_node)

time.sleep(2)
print("Admin mode after setting to ONLINE")

# CSP Control admin mode
#print(f"  CSP Control: {csp_control.adminMode}")
csp_ctl = widgets.Valid(
    value=not csp_control.adminMode,
    description="CSP Control online",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(csp_ctl)

# CSP Subarray admin mode
# print(f"  CSP Subarray: {csp_subarray.adminMode}")
csp_sub = widgets.Valid(
    value=not csp_subarray.adminMode,
    description="CSP Subarray online",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(csp_sub)

# CBF Controller admin mode
# print(f"  CBF Controller: {cbf_controller.adminMode}")
cbf_ctl = widgets.Valid(
    value=not cbf_controller.adminMode,
    description="CBF Controller online",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(cbf_ctl)

# CBF Subarray
# print(f"  CBF Subarray: {cbf_subarray.adminMode}")
cbf_sub = widgets.Valid(
    value=not cbf_subarray.adminMode,
    description="CBF Subarray online",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(cbf_sub)

# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.cbfSimulationMode = 0
csp_control.commandTimeout = 99

time.sleep(2)
# print("\nChecking CBF Simulation Mode and CBF Timeout:")

# CBF Simulation Mode
# print(f"  CBF Simulation Mode: {bool(csp_control.cbfSimulationMode)}")
cbf_sim = widgets.Valid(
    value=bool(csp_control.cbfSimulationMode),
    description="CBF Simulation Mode",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(cbf_sim)

# CBF Timeout
# print(f"  CBF Timeout: {csp_control.commandTimeout} sec")
cbf_tim = widgets.IntSlider(
    value=csp_control.commandTimeout,
    min=0,
    max=180,
    step=1,
    description='CBF Timeout',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(cbf_tim)

In [ ]:
# Load the Dish Vcc Config / Init Sys Params
from IPython.display import JSON
csp_control.commandTimeout = cbf_tim.value

with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][0] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"
print(f"Dish config json file contents: \n{dish_config_json}")

nmax = 2
cfgp = widgets.IntProgress(
    value=0,
    min=0,
    max=nmax,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(cfgp)

# NOTE: RUN TWICE
for n in range(1, nmax+1):
    cfgp.value = n
    # print(f"\nLoad dish configuration {n}")
    tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))
    time.sleep(2)

vcc_cfg = json.loads(tmc_csp_master.dishVccConfig)
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{json.dumps(tmc_csp_master.dishVccConfig, indent=4)}")
print(f'TMC CSP Master Dish Vcc Config interface\n{vcc_cfg["interface"]}')
# json.dumps(tmc_csp_master.dishVccConfig, indent=4)
# cfgj = JSON(json.loads(tmc_csp_master.dishVccConfig), expanded=True)
# display(cfgj)

print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{tmc_csp_master.sourceDishVccConfig}"
)

In [ ]:
# Turn the Telescope On
import tango

print("Running the TelescopeOn command")
try:
    tmc_central_node.TelescopeOn()
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    print(f"Could not run command: {err_msg}")
print("Ready")

In [ ]:
# Check if telescope is on
p = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(p)
n = 0
while int(tmc_central_node.telescopeState) != 0:
    # print("Waiting for Telescope to come ON")
    n += 1
    p.value = n
    if n >= 100:
        break
    try:
        time.sleep(3)
    except KeyboardInterrupt:
        print("Abort")
        break
    
if int(tmc_central_node.telescopeState) == 0:
    print("Telecope is ON")
else:
    print("Telecope is not ON")

In [ ]:
# Assign Resources
if int(tmc_central_node.telescopeState) == 0:
    print("Telecope is ON")
else:
    print("Telecope is not ON")

print(f"SDP subarray is {sdp_subarray.state()}")

time.sleep(3)

print(
    "Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned"
)

with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = RECEPTORS
    assign_resources_json["sdp"]["resources"]["receptors"] = RECEPTORS


print(f"\nassign_resources_json file contents: \n{assign_resources_json}")

tmc_subarray.AssignResources(json.dumps(assign_resources_json))

time.sleep(2)
print(f"\nCBF Subarray Observation State: {tmc_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")

In [ ]:
from astropy.time import Time
from tango import DeviceProxy

dish_deployments = [f"tango://tango-databaseds.{SKA001_NAMESPACE}.svc.miditf.internal.skao.int:10000/mid-dish/dish-manager/SKA001", 
                    f"tango://tango-databaseds.{SKA036_NAMESPACE}.svc.miditf.internal.skao.int:10000/mid-dish/dish-manager/SKA036"]


def get_tai_from_unix_s(unix_s: float) -> float:
    """
    Calculate atomic time in seconds from unix time in seconds.

    :param unix_s: Unix time in seconds

    :return: atomic time (tai) in seconds
    """
    astropy_time_utc = Time(unix_s, format="unix")
    return astropy_time_utc.unix_tai


for dish in dish_deployments:
    dish_manager_proxy = DeviceProxy(dish)
    current_pointing = dish_manager_proxy.achievedPointing
    current_az = current_pointing[1]
    current_el = current_pointing[2]

    current_time_tai_s = get_tai_from_unix_s(time.time() + 120)

    # Directions to move values
    az_dir = 1 if current_az < 350 else -1
    el_dir = 1 if current_el < 80 else -1

    track_table = [
        current_time_tai_s + 3,
        current_az + 1 * az_dir,
        current_el + 1 * el_dir,
        current_time_tai_s + 5,
        current_az + 2 * az_dir,
        current_el + 2 * el_dir,
        current_time_tai_s + 7,
        current_az + 3 * az_dir,
        current_el + 3 * el_dir,
        current_time_tai_s + 9,
        current_az + 4 * az_dir,
        current_el + 4 * el_dir,
        current_time_tai_s + 11,
        current_az + 5 * az_dir,
        current_el + 5 * el_dir,
    ]

    dish_manager_proxy.programTrackTable = track_table

In [ ]:
# Configure Scan¶
print("Running the Configure command: subarray obsstate should go to Ready")

with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    configure_scan_json = json.load(f)

print(f"\nconfigure_scan_json file contents: \n{configure_scan_json}")

print(json.dumps(configure_scan_json))

tmc_subarray.Configure(json.dumps(configure_scan_json))
    
time.sleep(12)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_scan = d["sdp"]
    print(sdp_scan)
    sdp_subarray_leaf_node_dp.scan(json.dumps(sdp_scan))

sleep(10)
print(sdp_subarray_leaf_node_dp.sdpSubarrayObsState)

with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    print(csp_scan)
    csp_subarray_leaf_node_dp.scan(json.dumps(csp_scan))

sleep(2)
print(csp_subarray_leaf_node_dp.cspSubarrayObsState)

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
print("Running the End Scan command: subarray obsstate should go to Ready state")

sdp_subarray_leaf_node_dp.EndScan()
sleep(2)
csp_subarray_leaf_node_dp.EndScan()
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
print("Running the End command: subarray obsstate should go to Idle state")

sdp_subarray_leaf_node_dp.End()
sleep(2)
csp_subarray_leaf_node_dp.End()

sleep(2)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

sdp_subarray_leaf_node_dp.ReleaseAllResources()
sleep(2)
csp_subarray_leaf_node_dp.ReleaseAllResources()

sleep(2)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

In [ ]:
# Telescope Off
print("Running the TelescopeOff command")

tmc_central_node.TelescopeOff()

sleep(5)
print(f"TMC Central Node State: {tmc_central_node.State()}")
print(f"CSP Control State: {csp_control.State()}")
print(f"CBF Controller State: {cbf_controller.State()}")